In [ ]:
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
dataset = load_dataset("cnn_dailymail", '3.0.0')

# Load the tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 128

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=MAX_OUTPUT_LENGTH, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save the model
model_path = "./t5_summarizer_model"
trainer.save_model(model_path)
print(f"Model saved to {model_path}")

# Save the tokenizer
tokenizer_path = "./t5_summarizer_tokenizer"
tokenizer.save_pretrained(tokenizer_path)
print(f"Tokenizer saved to {tokenizer_path}")

In [ ]:
#Tuning using local GPU

import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

# Load the dataset
dataset = load_dataset("cnn_dailymail", '3.0.0')

# Load the tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

MAX_INPUT_LENGTH = 512
MAX_OUTPUT_LENGTH = 128

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=MAX_OUTPUT_LENGTH, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Increased batch size for GPU
    per_device_eval_batch_size=8,   # Increased batch size for GPU
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=True,  # Enable mixed precision training
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save the model
model_path = "./t5_summarizer_model"
trainer.save_model(model_path)
print(f"Model saved to {model_path}")

# Save the tokenizer
tokenizer_path = "./t5_summarizer_tokenizer"
tokenizer.save_pretrained(tokenizer_path)
print(f"Tokenizer saved to {tokenizer_path}")

In [ ]:
dataset = load_dataset("cnn_dailymail",'3.0.0')
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

MAX_INPUT_LENGTH = 1024
MAX_OUTPUT_LENGTH = 128

def pre_process_fn(inputs):
    articles = [article.replace('/n',' ') for article in inputs['article']]
    highlights = [highlight.replace('/n',' ') for highlight in inputs['highlights']]

    model_inputs = tokenizer(articles,max_length = MAX_INPUT_LENGTH,padding='max_length',truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(highlights,max_length=MAX_INPUT_LENGTH,padding='max_length',truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(pre_process_fn,batched=True)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base',quantization_config=quantization_config,device_map='auto')
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16, #Rank of the matrix
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

# Wrap the model with LoRA
model = get_peft_model(model, config)

trainer_args = Seq2SeqTrainingArguments(
    output_dir="./results",               # Directory to save the model and logs
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    learning_rate=2e-5,                   # Learning rate for the optimizer
    per_device_train_batch_size=4,        # Batch size for training
    per_device_eval_batch_size=4,         # Batch size for evaluation
    weight_decay=0.01,                    # Weight decay for regularization
    save_total_limit=3,                   # Limit the total amount of checkpoints
    predict_with_generate=True,           # Use generation for evaluation
    num_train_epochs=1,                   # Number of training epochs
    logging_steps=10,                     # Log every 10 steps
    logging_dir='./logs',                 # Directory to save logs
    fp16 = True,
    optim = "paged_adamw_32bit",
    gradient_accumulation_steps = 4,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = trainer_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    tokenizer = tokenizer,
)

trainer.train() #gives an error saying element 0 of tensors does not require grad and does not have a grad_fn